In [1]:
import data_preprocessing as dp
import neural_network_library as nnl
import numpy as np
import matplotlib.pyplot as plt

# set the random seed
np.random.seed(42)

In [2]:
X_train, X_val, y_train, y_val, X_test, y_test = dp.preprocess_data()

In [ ]:
# Model configuration
model = nnl.Sequential([
    nnl.Linear(11, 64),
    nnl.ReLU(),
    nnl.Linear(64, 64),
    nnl.ReLU(),
    nnl.Linear(64, 64),
    nnl.ReLU(),
    nnl.Linear(64, 1)
])
loss = nnl.MseLoss()

# Training parameters
learning_rate = 0.001
batch_size = 256
n_epochs = 200
patience = 3
best_val_loss = np.inf
patience_counter = 0
train_losses, val_losses = [], []

# Training loop with early stopping
for epoch in range(n_epochs):
    # Shuffle training data
    shuffled_idx = np.random.permutation(X_train.shape[0])
    X_shuffled = X_train[shuffled_idx]
    y_shuffled = y_train[shuffled_idx]

    epoch_losses = []

    # loop through data in batches
    for step in range(0, X_shuffled.shape[0], batch_size):
        end_idx = step + batch_size
        X_batch = X_shuffled[step:end_idx]
        y_batch = y_shuffled[step:end_idx]
        
        # Forward pass
        y_pred = model.forward(X_batch)
        batch_loss = loss.forward(y_pred, y_batch)
        epoch_losses.append(batch_loss)
        
        # Backward pass
        grad = loss.backward()
        model.backward(grad)
        
        # Update weights
        for layer in model.layers:
            if isinstance(layer, nnl.Linear):
                layer.weights -= learning_rate * layer.grad_weights
                layer.bias -= learning_rate * layer.grad_bias
    
    # avg training loss for epoch
    avg_train_loss = np.mean(epoch_losses)
    train_losses.append(avg_train_loss)
    
    val_pred = model.forward(X_val)
    val_loss = loss.forward(val_pred, y_val)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Patience: {patience_counter}/{patience}")

    # Early stopping check
    if round(val_loss, 4) <= round(best_val_loss, 4):
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break   

In [ ]:
# Final evaluation
test_pred = model.forward(X_test).flatten() #np.expm1(model.forward(X_test).flatten())
rmse = np.sqrt(np.mean((y_test - test_pred)**2))
mae = np.mean(np.abs(y_test - test_pred))
rmsle = np.sqrt(np.mean((np.log1p(y_test) - np.log1p(test_pred))**2))
print(f"\nTest RMSLE: {rmsle:.4f}")
print(f"\nTest RMSE: {rmse:.2f}")
print(f"Test MAE: {mae:.2f}")

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training History')
plt.legend()
plt.show()

In [14]:
test_pred = np.expm1(model.forward(X_test).flatten())

In [ ]:
test_pred

In [ ]:
rmsle = np.sqrt(np.mean((np.log1p(y_test) - np.log1p(test_pred))**2))
print(f"\nTest RMSLE: {rmsle:.4f}")